In [1]:
import os

In [2]:
os.chdir('../')

In [ ]:
def TimeAtRisk(X, cohort, time_at_risk = 0):
    target_cohort = cohort[cohort['cohort_definition_id']==1]
    outcome_cohort = cohort[cohort['cohort_definition_id']==2]
    cohort_at_risk = target_cohort.merge(outcome_cohort, how='inner', on='subject_id')
    if time_at_risk != 0:
        cohort_at_risk['time_to_event'] = cohort_at_risk['cohort_start_date_y'] \
                                          - cohort_at_risk['cohort_start_date_x']
        cohort_at_risk = cohort_at_risk[cohort_at_risk['time_to_event'].dt.days < time_at_risk]
    cohort_at_risk = cohort_at_risk.subject_id.unique().tolist()
    print('Subject at risk:'+ str(len(cohort_at_risk)))
    X['target'] = 1
    X['target'] = X['target'].where(X['person_id'].isin(cohort_at_risk), 0)
    print('Number of patients in final X: '+str(len(X)))
    return X

In [ ]:
class PredictionModels():
    def __init__(self, df):
        a = df[df['target']==1]
        b = df[df['target']==0]
        print('Subjects in target=1: '+str(len(a)))
        print('Subjects in target=0: '+str(len(b)))
        
        X = df.drop('target', axis=1)
        y = df['target']
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.25)
        
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        
        #if (len(a)+len(b))/20 < abs(len(b)-len(a)):
        #    print('Imbalanced dataset: resampling train set')
        #    print('************************************')
        #    c = math.trunc((max(len(a), len(b)) + min(len(a), len(b)))/2)
        #    a = self.X_train.sample(n=c, replace=True, random_state=3)
        #    b = self.y_train.sample(n=c, replace=True, random_state=3)
        #    df = pd.concat([a, b], ignore_index=True)

    def __call__(self):
        classifiers = [
            DummyClassifier(strategy='most_frequent', random_state=0),
            KNeighborsClassifier(3),
            SVC(kernel="rbf", C=0.025, probability=True),
            #NuSVC(probability=True),
            DecisionTreeClassifier(),
            RandomForestClassifier(),
            AdaBoostClassifier(),
            GradientBoostingClassifier()
            ]

        for classifier in classifiers:
            model = classifier.fit(self.X_train, self.y_train)
            y_pred = model.predict(self.X_test)
            print(classifier)
            print("Training score: %.3f" % model.score(self.X_train, self.y_train))
            print("Test score: %.3f" % accuracy_score(self.y_test, y_pred))
            print('*** Confusion matrix ***')
            print(confusion_matrix(self.y_test, y_pred))
            print('*** Classification report ***')
            print(classification_report(self.y_test, y_pred))
            print('************************************')
